In [12]:
# Auxiliary Classifier GAN (ACGAN) Implementation
# ================================================
# This notebook implements ACGAN using TensorFlow/Keras.
# ACGAN generates class-conditional images by combining adversarial and classification tasks.
# Dataset used: MNIST (handwritten digits 0-9)





In [13]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt


In [14]:
# Step 1: Define hyperparameters for the model
NOISE_DIM = 100  # Dimension of the random noise vector
NUM_CLASSES = 10  # Number of classes in the dataset (0-9 for MNIST)
IMG_SHAPE = (28, 28, 1)  # Shape of the input image
BATCH_SIZE = 64  # Number of samples per training batch
EPOCHS = 50  # Number of training epochs
BUFFER_SIZE = 60000  # Buffer size for shuffling data
LEARNING_RATE = 0.0002  # Learning rate for optimizers


In [15]:
# Step 2: Load and preprocess the MNIST dataset
def preprocess_data():
    """
    Loads the MNIST dataset and normalizes the pixel values to [-1, 1].
    Also adds a channel dimension to make the images compatible with Conv2D layers.
    """
    (x_train, y_train), (_, _) = tf.keras.datasets.mnist.load_data()
    x_train = (x_train.astype(np.float32) - 127.5) / 127.5  # Normalize to [-1, 1]
    x_train = np.expand_dims(x_train, axis=-1)  # Add a channel dimension
    return x_train, y_train

In [57]:
def preprocess_data():
    (x_train, y_train), (_, _) = tf.keras.datasets.mnist.load_data()
    x_train = (x_train.astype(np.float32) - 127.5) / 127.5  # Normalize to [-1, 1]
    x_train = np.expand_dims(x_train, axis=-1)  # Add channel dimension
    return x_train, y_train


In [58]:
# Step 3: Define the Generator model
def build_generator():
    """
    Creates the Generator model.
    The generator takes a noise vector and a class label as inputs
    and generates an image corresponding to the given class.
    """
    # Noise input
    noise = layers.Input(shape=(NOISE_DIM,))
    # Class label input
    label = layers.Input(shape=(1,))

    # Embed the label into a vector of size NOISE_DIM
    label_embedding = layers.Embedding(NUM_CLASSES, NOISE_DIM)(label)
    label_embedding = layers.Flatten()(label_embedding)

    # Combine noise and label embeddings
    input = layers.Multiply()([noise, label_embedding])

    # Fully connected layer to project and reshape input
    x = layers.Dense(7 * 7 * 256, activation="relu")(input)
    x = layers.Reshape((7, 7, 256))(x)

    # Upsampling using transposed convolutions
    x = layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding="same", activation="relu")(x)
    x = layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding="same", activation="relu")(x)
    x = layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding="same", activation="tanh")(x)

    return tf.keras.Model([noise, label], x, name="Generator")

In [ ]:

# # Step 4: Define the Discriminator model
# def build_discriminator():
#     """
#     Creates the Discriminator model.
#     The discriminator takes an image as input and predicts:
#     1. Whether the image is real or fake (binary classification).
#     2. The class label of the image (multi-class classification).
#     """
#     img = layers.Input(shape=IMG_SHAPE)

#     # Convolutional layers for feature extraction
#     x = layers.Conv2D(64, (5, 5), strides=(2, 2), padding="same", activation="relu")(img)
#     x = layers.Conv2D(128, (5, 5), strides=(2, 2), padding="same", activation="relu")(x)
#     x = layers.Flatten()(x)

#     # Real/Fake classification output
#     real_fake = layers.Dense(1, activation="sigmoid", name="real_fake")(x)

#     # Class label prediction output
#     class_label = layers.Dense(NUM_CLASSES, activation="softmax", name="class_label")(x)

#     return tf.keras.Model(img, [real_fake, class_label], name="Discriminator")




In [60]:
IMG_HEIGHT = 32  # Height of input images
IMG_WIDTH = 32   # Width of input images
IMG_CHANNELS = 3 # Number of color channels (3 for RGB, 1 for grayscale)
IMG_SHAPE = (28, 28, 1)  # Shape of MNIST images



In [61]:
def build_discriminator():
    img = layers.Input(shape=IMG_SHAPE)  # Use MNIST image shape

    x = layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same")(img)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same")(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Flatten()(x)
    x = layers.Dense(128)(x)
    x = layers.LeakyReLU(0.2)(x)

    # Outputs for real/fake classification and class label prediction
    validity = layers.Dense(1, activation="sigmoid", name="validity")(x)
    label = layers.Dense(NUM_CLASSES, activation="softmax", name="label")(x)

    return tf.keras.Model(img, [validity, label], name="discriminator")


NameError: name 'x' is not defined

In [68]:
# Step 5: Compile the models

def compile_models(generator, discriminator):
    """
    Compiles the Generator, Discriminator, and the combined GAN model.
    """
    # # Loss functions
    # adversarial_loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
    # classification_loss = tf.keras.losses.SparseCategoricalCrossentropy()

    # Optimizer
    optimizer = tf.keras.optimizers.Adam(LEARNING_RATE, beta_1=0.5)

    # # Compile the discriminator
    # discriminator.compile(
    #     loss=[adversarial_loss, classification_loss],
    #     optimizer=optimizer,
    #     metrics=["accuracy"],  # General metrics for all outputs
    # )

    adversarial_loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
    classification_loss = tf.keras.losses.SparseCategoricalCrossentropy()

    discriminator.compile(
        loss=[adversarial_loss, classification_loss],
        optimizer=tf.keras.optimizers.Adam(0.0002, beta_1=0.5),
        metrics=["accuracy"]  # Optional metrics for debugging
)

    # Freeze discriminator weights for combined model training
    discriminator.trainable = False

    # Combined GAN model
    noise = layers.Input(shape=(NOISE_DIM,))
    label = layers.Input(shape=(1,))
    img = generator([noise, label])  # Generated image
    valid, target_label = discriminator(img)  # Discriminator outputs

    combined = tf.keras.Model([noise, label], [valid, target_label])
    combined.compile(
        loss=[adversarial_loss, classification_loss], optimizer=optimizer
    )

    return generator, discriminator, combined



In [73]:
# Step 6: Train the models
def train(generator, discriminator, combined, x_train, y_train):
    """
    Trains the ACGAN models by alternating between:
    1. Training the discriminator on real and fake images.
    2. Training the generator to fool the discriminator and generate images with correct labels.
    """
    # Create TensorFlow dataset
    dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

    for epoch in range(EPOCHS):
        for real_images, labels in dataset:
            batch_size = real_images.shape[0]

            # Generate fake images
            noise = np.random.normal(0, 1, (batch_size, NOISE_DIM))
            fake_labels = np.random.randint(0, NUM_CLASSES, batch_size)
            fake_images = generator.predict([noise, fake_labels])

            # Labels for discriminator
            valid = np.ones((batch_size, 1))  # Real images -> "valid"
            fake = np.zeros((batch_size, 1))  # Fake images -> "fake"

            # Train discriminator
            d_loss_real = discriminator.train_on_batch(real_images, [valid, labels])
            d_loss_fake = discriminator.train_on_batch(fake_images, [fake, fake_labels])
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train generator
            noise = np.random.normal(0, 1, (batch_size, NOISE_DIM))
            fake_labels = np.random.randint(0, NUM_CLASSES, batch_size)
            valid = np.ones((batch_size, 1))  # Generator tries to produce "real"

            g_loss = combined.train_on_batch([noise, fake_labels], [valid, fake_labels])

        # Print progress
        print(
            f"Epoch {epoch + 1}/{EPOCHS} - D loss: {d_loss[0]:.4f}, G loss: {g_loss[0]:.4f}"
        )

        # Save samples every few epochs
        if (epoch + 1) % 10 == 0:
            save_samples(generator, epoch + 1)


In [70]:
# Step 7: Visualize generated images
def save_samples(generator, epoch):
    """
    Generates and saves a grid of images for each class (0-9).
    """
    noise = np.random.normal(0, 1, (NUM_CLASSES, NOISE_DIM))
    labels = np.arange(NUM_CLASSES)
    generated_images = generator.predict([noise, labels])

    # Rescale images from [-1, 1] to [0, 1]
    generated_images = 0.5 * generated_images + 0.5

    # Plot and save
    fig, axs = plt.subplots(1, NUM_CLASSES, figsize=(20, 4))
    for i in range(NUM_CLASSES):
        axs[i].imshow(generated_images[i].squeeze(), cmap="gray")
        axs[i].axis("off")
        axs[i].set_title(f"Digit: {i}")
    plt.savefig(f"acgan_generated_epoch_{epoch}.png")
    plt.show()

In [71]:
def train(generator, discriminator, combined, x_train, y_train):
    for epoch in range(EPOCHS):
        for real_images, labels in dataset:
            batch_size = real_images.shape[0]

            # Real/Fake labels
            valid = np.ones((batch_size, 1))  # Shape (batch_size, 1)
            fake = np.zeros((batch_size, 1))  # Shape (batch_size, 1)

            # Generate fake images
            noise = np.random.normal(0, 1, (batch_size, NOISE_DIM))
            fake_labels = np.random.randint(0, NUM_CLASSES, batch_size)
            fake_images = generator.predict([noise, fake_labels])

            # Train discriminator
            d_loss_real = discriminator.train_on_batch(real_images, [valid, labels])
            d_loss_fake = discriminator.train_on_batch(fake_images, [fake, fake_labels])
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train generator
            valid = np.ones((batch_size, 1))  # Generator tries to produce "real"
            g_loss = combined.train_on_batch([noise, fake_labels], [valid, fake_labels])

        print(f"Epoch {epoch}/{EPOCHS} - D loss: {d_loss[0]:.4f}, G loss: {g_loss[0]:.4f}")


In [74]:
# Step 8: Run the ACGAN
if __name__ == "__main__":
    x_train, y_train = preprocess_data()  # Load data
    generator = build_generator()  # Create generator
    discriminator = build_discriminator()  # Create discriminator
    generator, discriminator, combined = compile_models(generator, discriminator)
    train(generator, discriminator, combined, x_train, y_train)  # Train models

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step 


c:\Users\Rishu\AppData\Local\anaconda3\envs\rexxes\lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_277', 'keras_tensor_278']. Received: the structure of inputs=('*', '*')
  warnings.warn(


AttributeError: 'NoneType' object has no attribute 'update_state'